In [1]:
import cv2
import numpy as np
from scipy.ndimage import label

In [ ]:
def split_image(image_path, tile_size=(2000, 2000)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    h, w = image.shape
    tiles = []
    
    for i in range(0, w, tile_size[0]):
        tile = image[:, i:i + tile_size[0]]
        tiles.append((tile, i))  # Сохраняем координату x для корректного объединения границ
    
    return tiles

def detect_objects(image):
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    structure = np.ones((3,3), dtype=np.int)
    labeled, num_objects = label(binary, structure)
    
    objects = []
    for i in range(1, num_objects + 1):
        y, x = np.where(labeled == i)
        x_min, x_max = x.min(), x.max()
        y_min, y_max = y.min(), y.max()
        objects.append((x_min, y_min, x_max, y_max))
    
    return objects

def process_image(image_path):
    tiles = split_image(image_path)
    all_objects = []
    
    for tile, x_offset in tiles:
        objects = detect_objects(tile)
        adjusted_objects = [(x + x_offset, y, x_max + x_offset, y_max) for x, y, x_max, y_max in objects]
        all_objects.extend(adjusted_objects)
    
    return all_objects

def draw_objects(image_path, objects, output_path="output.bmp"):
    image = cv2.imread(image_path)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, image)



In [ ]:
# Пример использования
image_path = "input.bmp"
objects = process_image(image_path)
draw_objects(image_path, objects)
